In [1]:
import requests
import pandas as pd
import numpy as np
import random

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [6]:
CLIENT_ID = '4XJERW0UXHYGTBU0UDD5VUXEVQCLVXXV4ZUDI51YICLHWTOC'
CLIENT_SECRET = 'YAJY3B3BOGU3TG2H1M1DQA2HSD40M2DVKKMNCCEDBWYHB3CN'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4XJERW0UXHYGTBU0UDD5VUXEVQCLVXXV4ZUDI51YICLHWTOC
CLIENT_SECRET:YAJY3B3BOGU3TG2H1M1DQA2HSD40M2DVKKMNCCEDBWYHB3CN


In [2]:
HMS_address = '25 Shattuck Street, Boston, MA'


geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(HMS_address)
HMS_latitude = location.latitude
HMS_longitude = location.longitude
print("Harvard Medical School:", HMS_latitude, HMS_longitude)

Harvard Medical School: 42.33564455 -71.1042236537206


In [4]:
search_query = 'Athletic Track'
radius = 500
print(search_query + ' .... OK!')

Athletic Track .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, HMS_latitude, HMS_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4XJERW0UXHYGTBU0UDD5VUXEVQCLVXXV4ZUDI51YICLHWTOC&client_secret=YAJY3B3BOGU3TG2H1M1DQA2HSD40M2DVKKMNCCEDBWYHB3CN&ll=42.33564455,-71.1042236537206&v=20180604&query=Athletic Track&radius=500&limit=30'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d8faa06a6ec9800392f374a'},
 'response': {'venues': [{'id': '4ed67a5d9adf5095962f1974',
    'name': 'Vanderbilt Hall Athletic Facility',
    'location': {'address': '107 Avenue Louis Pasteur',
     'crossStreet': 'at Longwood Ave',
     'lat': 42.337717420947406,
     'lng': -71.10377389250911,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.337717420947406,
       'lng': -71.10377389250911}],
     'distance': 233,
     'postalCode': '02115',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['107 Avenue Louis Pasteur (at Longwood Ave)',
      'Boston, MA 02115',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1569

In [12]:
venues = results['response']['venues']

HMS_Data = json_normalize(venues)
HMS_Data.head

<bound method NDFrame.head of                                           categories  hasPerk  \
0  [{'id': '4bf58dd8d48988d176941735', 'name': 'G...    False   

                         id          location.address location.cc  \
0  4ed67a5d9adf5095962f1974  107 Avenue Louis Pasteur          US   

  location.city location.country location.crossStreet  location.distance  \
0        Boston    United States      at Longwood Ave                233   

                           location.formattedAddress  \
0  [107 Avenue Louis Pasteur (at Longwood Ave), B...   

                             location.labeledLatLngs  location.lat  \
0  [{'label': 'display', 'lat': 42.33771742094740...     42.337717   

   location.lng location.postalCode location.state  \
0    -71.103774               02115             MA   

                                name    referralId  
0  Vanderbilt Hall Athletic Facility  v-1569696262  >

In [13]:
filtered_columns = ['name', 'categories'] + [col for col in HMS_Data.columns if col.startswith('location.')] + ['id']
HMS_Data_filtered = HMS_Data.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

HMS_Data_filtered['categories'] = HMS_Data_filtered.apply(get_category_type, axis=1)

HMS_Data_filtered.columns = [column.split('.')[-1] for column in HMS_Data_filtered.columns]

HMS_Data_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Vanderbilt Hall Athletic Facility,Gym,107 Avenue Louis Pasteur,US,Boston,United States,at Longwood Ave,233,"[107 Avenue Louis Pasteur (at Longwood Ave), B...","[{'label': 'display', 'lat': 42.33771742094740...",42.337717,-71.103774,02115,MA,4ed67a5d9adf5095962f1974


In [14]:
HMS_Data_filtered.name

0    Vanderbilt Hall Athletic Facility
Name: name, dtype: object

In [21]:
HMS_Data_filtered.distance

0    233
Name: distance, dtype: int64

In [15]:
venues_map = folium.Map(location=[HMS_latitude, HMS_longitude], zoom_start=13)

folium.features.CircleMarker(
    [HMS_latitude, HMS_longitude],
    radius=10,
    color='red',
    popup='Havard Medical School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(HMS_Data_filtered.lat, HMS_Data_filtered.lng, HMS_Data_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [16]:
venue_id = '4ed67a5d9adf5095962f1974' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4ed67a5d9adf5095962f1974?client_id=4XJERW0UXHYGTBU0UDD5VUXEVQCLVXXV4ZUDI51YICLHWTOC&client_secret=YAJY3B3BOGU3TG2H1M1DQA2HSD40M2DVKKMNCCEDBWYHB3CN&v=20180604'

In [17]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


{'id': '4ed67a5d9adf5095962f1974',
 'name': 'Vanderbilt Hall Athletic Facility',
 'contact': {'phone': '6174321630',
  'formattedPhone': '(617) 432-1630',
  'twitter': 'harvardmed'},
 'location': {'address': '107 Avenue Louis Pasteur',
  'crossStreet': 'at Longwood Ave',
  'lat': 42.337717420947406,
  'lng': -71.10377389250911,
  'labeledLatLngs': [{'label': 'display',
    'lat': 42.337717420947406,
    'lng': -71.10377389250911}],
  'postalCode': '02115',
  'cc': 'US',
  'city': 'Boston',
  'state': 'MA',
  'country': 'United States',
  'formattedAddress': ['107 Avenue Louis Pasteur (at Longwood Ave)',
   'Boston, MA 02115',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/vanderbilt-hall-athletic-facility/4ed67a5d9adf5095962f1974',
 'categories': [{'id': '4bf58dd8d48988d176941735',
   'name': 'Gym',
   'pluralName': 'Gyms',
   'shortName': 'Gym',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
    'suffix': '.png'},
   'primary': True}],


In [20]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [25]:
HMS_address = '630 W 168th St, New York, NY 10032'


geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(HMS_address)
CMS_latitude = location.latitude
CMS_longitude = location.longitude
print("Columbia Medical School:", CMS_latitude, CMS_longitude)

Columbia Medical School: 40.8416764 -73.9415796


In [26]:
search_query = 'Athletic Track'
radius = 500
print(search_query + ' .... OK!')

Athletic Track .... OK!


In [27]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, CMS_latitude, CMS_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4XJERW0UXHYGTBU0UDD5VUXEVQCLVXXV4ZUDI51YICLHWTOC&client_secret=YAJY3B3BOGU3TG2H1M1DQA2HSD40M2DVKKMNCCEDBWYHB3CN&ll=40.8416764,-73.9415796&v=20180604&query=Athletic Track&radius=500&limit=30'

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d8fb164475abd0039e58ae3'},
 'response': {'venues': [{'id': '4b5b0144f964a5205fde28e3',
    'name': 'New Balance Track & Field Center at The Armory',
    'location': {'address': '216 Fort Washington Ave',
     'crossStreet': 'near 168th & Broadway',
     'lat': 40.84237921810222,
     'lng': -73.94187469369459,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.84237921810222,
       'lng': -73.94187469369459}],
     'distance': 82,
     'postalCode': '10032',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['216 Fort Washington Ave (near 168th & Broadway)',
      'New York, NY 10032',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d106941735',
      'name': 'Track',
      'pluralName': 'Tracks',
      'shortName': 'Track',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_track_',
       'suffix': '.png'},


In [31]:
venues = results['response']['venues']

CMS_Data = json_normalize(venues)
CMS_Data.head

<bound method NDFrame.head of                                           categories  hasPerk  \
0  [{'id': '4bf58dd8d48988d106941735', 'name': 'T...    False   
1  [{'id': '4bf58dd8d48988d1b2941735', 'name': 'C...    False   
2  [{'id': '4bf58dd8d48988d16d941735', 'name': 'C...    False   
3  [{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...    False   
4  [{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...    False   

                         id         location.address location.cc  \
0  4b5b0144f964a5205fde28e3  216 Fort Washington Ave          US   
1  50e4986ae4b0167bf4bcb4a1             60 Haven Ave          US   
2  4bbca8c34e069c7443b19de3  216 Fort Washington Ave          US   
3  50d4e46ee4b0f4383822524b                      NaN          US   
4  4e15aaa5b61c42e7c54cf6c3  216 Fort Washington Ave          US   

  location.city location.country   location.crossStreet  location.distance  \
0      New York    United States  near 168th & Broadway                 82   
1      New Yor

In [33]:
filtered_columns = ['name', 'categories'] + [col for col in CMS_Data.columns if col.startswith('location.')] + ['id']
CMS_Data_filtered = CMS_Data.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

CMS_Data_filtered['categories'] = CMS_Data_filtered.apply(get_category_type, axis=1)

CMS_Data_filtered.columns = [column.split('.')[-1] for column in CMS_Data_filtered.columns]

CMS_Data_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,New Balance Track & Field Center at The Armory,Track,216 Fort Washington Ave,US,New York,United States,near 168th & Broadway,82,[216 Fort Washington Ave (near 168th & Broadwa...,"[{'label': 'display', 'lat': 40.84237921810222...",40.842379,-73.941875,10032,NY,4b5b0144f964a5205fde28e3
1,Bard Athletic Center,College Gym,60 Haven Ave,US,New York,United States,169th and Haven Ave,261,"[60 Haven Ave (169th and Haven Ave), New York,...","[{'label': 'display', 'lat': 40.84339125113941...",40.843391,-73.943707,10032,NY,50e4986ae4b0167bf4bcb4a1
2,Finish Line Cafe (@ New Balance Track and Fie...,Café,216 Fort Washington Ave,US,New York,United States,at West 168th St,73,"[216 Fort Washington Ave (at West 168th St), N...","[{'label': 'display', 'lat': 40.842305, 'lng':...",40.842305,-73.941863,10032,NY,4bbca8c34e069c7443b19de3
3,Armory Track & Field,Athletics & Sports,NaN,US,New York,United States,NaN,50,"[New York, NY 10032, United States]","[{'label': 'display', 'lat': 40.84212671215585...",40.842127,-73.941536,10032,NY,50d4e46ee4b0f4383822524b
4,Police Athletic League (P.A.L.),Athletics & Sports,216 Fort Washington Ave,US,New York,United States,169th St.,96,"[216 Fort Washington Ave (169th St.), New York...","[{'label': 'display', 'lat': 40.84253728743216...",40.842537,-73.941479,10032,NY,4e15aaa5b61c42e7c54cf6c3


In [34]:
venues_map = folium.Map(location=[CMS_latitude, CMS_longitude], zoom_start=13)

folium.features.CircleMarker(
    [CMS_latitude, CMS_longitude],
    radius=10,
    color='red',
    popup='Columbia Medical School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(CMS_Data_filtered.lat, CMS_Data_filtered.lng,CMS_Data_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [35]:
HMS_Data_filtered.name

0       New Balance Track & Field Center at The Armory
1                                 Bard Athletic Center
2    Finish Line Cafe  (@ New Balance Track and Fie...
3                                 Armory Track & Field
4                      Police Athletic League (P.A.L.)
Name: name, dtype: object

In [36]:
HMS_Data_filtered.distance

0     82
1    261
2     73
3     50
4     96
Name: distance, dtype: int64

In [58]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [79]:

LIMIT = 100
radius = 200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    CMS_latitude, 
    CMS_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=4XJERW0UXHYGTBU0UDD5VUXEVQCLVXXV4ZUDI51YICLHWTOC&client_secret=YAJY3B3BOGU3TG2H1M1DQA2HSD40M2DVKKMNCCEDBWYHB3CN&v=20180604&ll=40.8416764,-73.9415796&radius=200&limit=100'

In [80]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d8fbb416001bc002b212bba'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Washington Heights',
  'headerFullLocation': 'Washington Heights, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 40.8434764018, 'lng': -73.9392047228954},
   'sw': {'lat': 40.8398763982, 'lng': -73.9439544771046}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ac93103f964a52013bf20e3',
       'name': 'Highbridge Park',
       'location': {'address': '173rd St & Amsterdam Ave',
        'crossStreet': 'at Amsterdam Ave',
        'lat': 40.84076848482368,
        'lng': -73.94292688638392,
        'labe

In [81]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [82]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head

<bound method NDFrame.head of                                               name          categories  \
0                                  Highbridge Park                Park   
1   New Balance Track & Field Center at The Armory               Track   
2                                       Tasty Deli      Sandwich Place   
3       Fort Washington Greenmarket Farmers Market      Farmers Market   
4                                    Mike's Bagels          Bagel Shop   
5                               Shana's Nail & Spa          Nail Salon   
6                                  Famous Famiglia         Pizza Place   
7                           Chipotle Mexican Grill  Mexican Restaurant   
8                                         Coogan's                 Bar   
9                   Jade Garden Chinese Restaurant  Chinese Restaurant   
10                                       Starbucks         Coffee Shop   
11                                         Subways      Sandwich Place   

       

In [64]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

39 venues were returned by Foursquare.


In [85]:
import json

In [86]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [87]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [89]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [91]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [93]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head

<bound method NDFrame.head of            Borough         Neighborhood   Latitude  Longitude
0            Bronx            Wakefield  40.894705 -73.847201
1            Bronx           Co-op City  40.874294 -73.829939
2            Bronx          Eastchester  40.887556 -73.827806
3            Bronx            Fieldston  40.895437 -73.905643
4            Bronx            Riverdale  40.890834 -73.912585
5            Bronx          Kingsbridge  40.881687 -73.902818
6        Manhattan          Marble Hill  40.876551 -73.910660
7            Bronx             Woodlawn  40.898273 -73.867315
8            Bronx              Norwood  40.877224 -73.879391
9            Bronx       Williamsbridge  40.881039 -73.857446
10           Bronx           Baychester  40.866858 -73.835798
11           Bronx       Pelham Parkway  40.857413 -73.854756
12           Bronx          City Island  40.847247 -73.786488
13           Bronx         Bedford Park  40.870185 -73.885512
14           Bronx   University Heights 

In [96]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head

<bound method NDFrame.head of       Borough         Neighborhood   Latitude  Longitude
0   Manhattan          Marble Hill  40.876551 -73.910660
1   Manhattan            Chinatown  40.715618 -73.994279
2   Manhattan   Washington Heights  40.851903 -73.936900
3   Manhattan               Inwood  40.867684 -73.921210
4   Manhattan     Hamilton Heights  40.823604 -73.949688
5   Manhattan       Manhattanville  40.816934 -73.957385
6   Manhattan       Central Harlem  40.815976 -73.943211
7   Manhattan          East Harlem  40.792249 -73.944182
8   Manhattan      Upper East Side  40.775639 -73.960508
9   Manhattan            Yorkville  40.775930 -73.947118
10  Manhattan           Lenox Hill  40.768113 -73.958860
11  Manhattan     Roosevelt Island  40.762160 -73.949168
12  Manhattan      Upper West Side  40.787658 -73.977059
13  Manhattan       Lincoln Square  40.773529 -73.985338
14  Manhattan              Clinton  40.759101 -73.996119
15  Manhattan              Midtown  40.754691 -73.981669
1

In [98]:
HMS_address = '630 W 168th St, New York, NY 10032'


geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(HMS_address)
CMS_latitude = location.latitude
CMS_longitude = location.longitude
print("Columbia Medical School:", CMS_latitude, CMS_longitude)

Columbia Medical School: 40.8416764 -73.9415796
